In [1]:
import happybase
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib

In [1]:
from starbase import Connection


In [146]:
cf = 'gsmgarena'

In [2]:
base_url = 'https://www.gsmarena.com/acer_iconia_talk_s-8306.php'

In [117]:
r = urllib.urlopen(base_url).read()
soup = BeautifulSoup(r, 'html.parser')

In [118]:
body = soup.find(id="body")

In [119]:
specs = soup.find('div', id='specs-list')

In [120]:
tables = {}
i = 1
for table in specs.find_all('table'):
    feature = table.find('th')
    if not feature:
        continue
    
    if table.find('tr', class_='tr-hover'):
        del_rows = []
        for f in table.find_all('td', class_='ttl'):
            if f.text == u'\xa0':
                curr_row = f.parent
                prev_row = curr_row.find_previous_sibling()
                while True:
                    prev_row_key = prev_row.find('td', class_='ttl')
                    prev_row_value = prev_row.find('td', class_='nfo')
                    if prev_row_key.text == u'\xa0':
                        prev_row = prev_row_key.parent.find_previous_sibling()
                    else:
                        prev_row_value.insert(1, '^^^')
                        prev_row_value.insert(2, f.find_next_sibling().text)
                        break
                del_rows.append(curr_row)
        
        [del_row.decompose() for del_row in del_rows]
    
    feature_key = feature.text
    feature.decompose()
    if feature_key in tables:
        feature_key = '{0}_{1}'.format(feature_key, i)
        i += 1
    
    tables[feature_key] = table

In [145]:
table.put(b'row-key', {b'gsmgarena:{}'.format('1'): b'value1', b'gsmgarena:col2': b'value2'})

In [132]:
b'cf:{}'.format('dd')

'cf:dd'

In [125]:
d = pd.read_html(tables['Network'].prettify())[0]

In [12]:
import happybase

connection = happybase.Connection('192.168.56.101')

In [13]:
connection.open()

In [14]:
connection.tables()

['phone_specs']

In [159]:
table = connection.table('phone_specs')

In [8]:
connection.close()

In [162]:
import re

In [167]:
def normalize(text):
    strip = re.sub(r'[^A-Za-z0-9\s]+', '', text.strip())
    return sub(r"[\s]+", "_", strip)

In [160]:
with table.batch(transaction=True) as b:
    for feature, data_html in tables.iteritems():
        data = pd.read_html(data_html.prettify())[0]

        data_row = {}
        for index, row in data.iterrows():
            key = b'{cf}:{key}'.format(cf=cf, key=normalize(row[0]))
            value = b'{value}'.format(value=row[1])
            data_row[key] = value
        rowkey = b'{name}.{feature}'.format(name='ACER', feature=normalize(feature))
        b.put(rowkey, data_row)
    #data = pd.Series(data.value.values, index=conf.key).to_dict()

{'gsmgarena:Dimensions': '191.7 x 101 x 9.4 mm (7.55 x 3.98 x 0.37 in)', 'gsmgarena:SIM': 'Dual SIM (Micro-SIM/Nano-SIM)', 'gsmgarena:Weight': '260 g (9.17 oz)'}
ACER.Body
{'gsmgarena:Speed': 'HSPA 42.2/11.5 Mbps, LTE Cat4 150/50 Mbps', 'gsmgarena:2G_bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2', 'gsmgarena:4G_bands': 'LTE band 1(2100), 3(1800), 7(2600), 8(900), 20(800)  ^^^  LTE band 3(1800), 7(2600), 8(900), 28(700)', 'gsmgarena:GPRS': 'Yes', 'gsmgarena:3G_bands': 'HSDPA 850 / 1900 / 2100', 'gsmgarena:Technology': 'GSM / HSPA / LTE', 'gsmgarena:EDGE': 'Yes'}
ACER.Network
{'gsmgarena:Announced': '2016, August', 'gsmgarena:Status': 'Available. Released 2016, October'}
ACER.Launch
{'gsmgarena:Chipset': 'Mediatek MT8735', 'gsmgarena:CPU': 'Quad-core 1.3 GHz Cortex-A53', 'gsmgarena:OS': 'Android 6.0 (Marshmallow)', 'gsmgarena:GPU': 'Mali-T720MP2'}
ACER.Platform
{'gsmgarena:Card_slot': 'microSD, up to 128 GB (dedicated slot)', 'gsmgarena:Internal': '16/32 GB, 2 GB RAM'}
ACER.Memory